In [45]:
import requests
import json
import pandas as pd
import pickle
import fasttext
import neattext.functions as nfx

embeddings model

In [29]:
model = fasttext.load_model("fasttext_model_teds.bin")

fetch a scanR publication

In [47]:
url="https://scanr.enseignementsup-recherche.gouv.fr/api/scanr-publications/_search?filter=externalIds.type:doi"

In [48]:
response = requests.get(url, verify=False)

C:\Users\haallat\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scanr.enseignementsup-recherche.gouv.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
response

<Response [200]>

In [50]:
page=response.json()

In [51]:
page.get('hits').get('hits')[3].get('_source').get('title')

{'default': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions',
 'en': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions'}

In [52]:
externalIds=page.get('hits').get('hits')[3].get('_source').get('externalIds')
externalIds

[{'type': 'doi', 'id': '10.1191/0269215506cr929oa'}]

In [53]:
doi=[x for x in externalIds if x['type']=='doi'][0]['id']
doi

'10.1191/0269215506cr929oa'

In [37]:
doi='10.1016/j.jrurstud.2016.09.009'  #ex de publi tagué "verte"

In [54]:
url_OA=f"https://api.openalex.org/works?filter=doi:{doi}"
response_OA = requests.get(url_OA)
data = response_OA.json()

In [55]:
topics=data.get('results')[0].get('topics')
topics_names=' '.join([topic.get('display_name') for topic in topics])
topics_names

'Gait Analysis and Fall Prevention in Elderly Classification and Interventions for Cerebral Palsy Global Burden of Child Injury Prevention'

In [56]:
title=data.get('results')[0].get('title')
title

'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions'

In [57]:
locations=data.get('results')[0].get('locations')
if (locations!=[]):
    for k in range(len(locations)):
        if pd.isna(locations[k].get('source'))==False:
            if locations[k].get('source').get('type')=='journal':
                locations_names=locations[k].get('source').get('display_name')
                locations_id=locations[k].get('source').get('issn_l') 
                break
print(locations_names,locations_id)

Clinical Rehabilitation 0269-2155


In [58]:
test=f"{title} {topics_names} {locations_names} {locations_id}"

In [59]:
test

'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions Gait Analysis and Fall Prevention in Elderly Classification and Interventions for Cerebral Palsy Global Burden of Child Injury Prevention Clinical Rehabilitation 0269-2155'

0 -> pas vert
<br />
1 -> vert

In [60]:
model.predict(nfx.remove_stopwords(test.lower()))

(('__label__0',), array([0.99986589]))